In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
import collections
import seaborn as sns
nltk.download('rslp')
from nltk.stem import RSLPStemmer
import heapq as hp
from collections import Counter,OrderedDict
import math

[nltk_data] Downloading package rslp to /home/vinicius/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [2]:
data = pd.read_csv('./results.csv')

### 1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [6]:
#generate tokens from a document
def parse(doc):
    words = ''.join(str(v) for v in doc).lower()
    return RegexpTokenizer(r'[A-zÀ-ú\d]{4,}').tokenize(words)

# produces: term:[(doc, frequency)]
#build an index for a collection of documents
def build_index_with_frequency(docs):
    index = {}
    n = 0
    for text in docs.text:
        n +=1
        tokens = parse(text)
        for token in tokens:
            if (token in index):
                hasDoc = False
                for i in range(len(index[token])):
                    tup = index[token][i]
                    if (tup[0] == n):
                        index[token][i] = (n, tup[1] + 1)
                        hasDoc = True
                if (not hasDoc):
                    index[token].append((n,1))
            else:
                index[token] = [(n,1)]
    return index
index = build_index_with_frequency(data)

### 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [11]:
# produces: term:[(doc, frequency), idf]
#build an index for a collection of documents
def build_index_with_frequency_and_idf(docs):
    index = {}
    n = 0
    M = len(docs)
    for text in docs.text:
        n +=1
        tokens = parse(text)
        for token in tokens:
            if (token in index):
                hasDoc = False
                for i in range(len(index[token])):
                    tup = index[token][i]
                    if (tup[0] == n):
                        index[token][i] = (n, tup[1] + 1)
                        hasDoc = True
                if (not hasDoc):
                    index[token].append((n,1))
            else:
                index[token] = [(n,1)]
    
    for elem in index:
        k = len(index[elem])
        idf = math.log((M+1)/k)
        index[elem].append(idf)
    return index

index = build_index_with_frequency_and_idf(data)